In [69]:
from SPARQLWrapper import SPARQLWrapper, JSON, CONSTRUCT, TURTLE

import rdflib
import pandas as pd

# sparql endpoint (since rdflib is so slow, alternatively, you can load both the ontology and the INPUT_FILENAME into graphdb and query from there)
sparql_endpoint = "http://localhost:7200/repositories/ocedo-small"

OCEDO_FIlENAME = 'files/oced_ontology.ttl'
INPUT_FILENAME = 'input/2013_small.ttl'
OUTPUT_FILENAME = INPUT_FILENAME.replace(".ttl", "-integrated.ttl")
OBJECT_OBJECT_MAP_FILENAME = 'files/2013_object-object_mapping.csv' 
EVENT_OBJECT_MAP_FILENAME = 'files/2013_event-object_mapping.csv'

In [70]:
# prepare rdflib graph
constructed_graph = rdflib.Graph()

ocedo = rdflib.Namespace('https://w3id.org/ocedo/core#')
ocedd = rdflib.Namespace('https://w3id.org/ocedo/domain#')
ocedr = rdflib.Namespace('https://w3id.org/ocedo/resource/')
constructed_graph.bind('ocedo', ocedo)
constructed_graph.bind('ocedd', ocedd)
constructed_graph.bind('ocedr', ocedr)

# load OCED ontology
constructed_graph.parse(OCEDO_FIlENAME, format="turtle")

# load input graph
input_graph = rdflib.Graph()
input_graph.parse(INPUT_FILENAME, format="turtle")
constructed_graph += input_graph


In [71]:
# prepare construct query for event_object enhancement
# -- parameters needed: $object_type, $ocedd_class, $ocedd_relation
event_object_cq = """
prefix ocedo: <https://w3id.org/ocedo/core#>
prefix ocedd: <https://w3id.org/ocedo/domain#>
prefix ocedr: <https://w3id.org/ocedo/resource/>

CONSTRUCT {
    ?object a $ocedd_class .
    ?event $ocedd_relation ?object .
} 
WHERE {
    ?eo a ocedo:EventObject ;
    	ocedo:eo_event ?event ;
    	ocedo:eo_object ?object ;
    .
    ?object ocedo:object_type "$object_type" .
}
"""

# prepare construct query for object_object enhancement
# -- parameters needed: $object1_class, $object2_class, $object1_type, $object2_type, $ocedd_relation
object_object_cq = """
prefix ocedo: <https://w3id.org/ocedo/core#>
prefix ocedd: <https://w3id.org/ocedo/domain#>
prefix ocedr: <https://w3id.org/ocedo/resource/>

CONSTRUCT {
    ?object1 a $object1_class .
    ?object2 a $object2_class .
    ?object1 $ocedd_relation ?object2 .
} 
WHERE {
    ?eo1 a ocedo:EventObject ;
    	ocedo:eo_event ?event ;
    	ocedo:eo_object ?object1 ;
    .
    ?eo2 a ocedo:EventObject ;
    	ocedo:eo_event ?event ;
    	ocedo:eo_object ?object2 ;
    .
    ?object1 ocedo:object_type "$object1_type" .
    ?object2 ocedo:object_type "$object2_type" .
}
"""

In [72]:

def build_event_object_cq(object_type, ocedd_class, ocedd_relation):
    cq = event_object_cq
    cq = cq.replace("$object_type", object_type)
    cq = cq.replace("$ocedd_class", ocedd_class)
    cq = cq.replace("$ocedd_relation", ocedd_relation)
    return cq

def build_object_object_cq(object1_class, object2_class, object1_type, object2_type, ocedd_relation):
    cq = object_object_cq
    cq = cq.replace("$object1_class", object1_class)
    cq = cq.replace("$object2_class", object2_class)
    cq = cq.replace("$object1_type", object1_type)
    cq = cq.replace("$object2_type", object2_type)
    cq = cq.replace("$ocedd_relation", ocedd_relation)
    return cq

def run_construct_query_endpoint(cq):
    sparql = SPARQLWrapper(sparql_endpoint)
    sparql.setQuery(cq)
    sparql.setReturnFormat(TURTLE)
    sparql.setMethod(CONSTRUCT)
    results = sparql.queryAndConvert()

    result_graph = rdflib.Graph()
    result_graph.parse(data=results, format='ttl')
    return result_graph

def run_construct_query_rdflib(cq):
    result_graph = rdflib.Graph()
    result_graph = constructed_graph.query(cq)

    return result_graph


In [73]:

df = pd.read_csv(EVENT_OBJECT_MAP_FILENAME)
for index, row in df.iterrows():
    o_type = row["object_type"]
    o_class = row["ocedd_class"]
    o_relation = row["ocedd_relation"]
    cq_string = build_event_object_cq(o_type, o_class, o_relation)
    # print(cq_string)
    constructed_graph += run_construct_query_endpoint(cq_string)
    # constructed_graph += run_construct_query_rdflib(cq_string)



In [74]:
df = pd.read_csv(OBJECT_OBJECT_MAP_FILENAME)
for index, row in df.iterrows():
    object1_class = row["object1_class"]
    object2_class = row["object2_class"]
    object1_type = row["object1_type"]
    object2_type = row["object2_type"]
    ocedd_relation = row["ocedd_relation"]
    cq_string = build_object_object_cq(object1_class, object2_class, object1_type, object2_type, ocedd_relation)
    # print(cq_string)
    constructed_graph += run_construct_query_endpoint(cq_string)
    # constructed_graph += run_construct_query_rdflib(cq_string) # ==> not sure why, but it's really slow

In [75]:


constructed_graph.serialize(destination=OUTPUT_FILENAME, format='ttl')



<Graph identifier=Neec8bf7ec2aa4c6eb072bb4151fc9852 (<class 'rdflib.graph.Graph'>)>